###### 4_optimizer_practice.ipynb

## 옵티마이저 (Optimizer)
- 신경망이 학습하면서 '손실 함수의 결과값이 최소가 되도록' 가중치를 조정하는 알고리즘
- 종류
  1. Adam (Adaptive Moment Estimation)
      - Momentum + RMSprop 조합
      - 거의 모든 문제에서 좋은 성능
      - 이미지, 시계열, 구조적 데이터 모두 가능
  2. SGD (Stochastic Gradient Descent)
      - 가장 기본적인 경사 하강법. 전체 데이터 대신 일부를 이용해 경사를 계산.
      - 구현이 단순하지만 학습률 선택이 중요
      - 학습용, 매우 단순한 모델에서 사용
  3. RMSprop (Root Mean Square Propagation)
      - 각 가중치마다 최근 변화량을 평균내어 학습률을 조정하는 방식
      - RNN, LSTM 같은 순차 모델에서 매우 안정적
      - 시계열, 자연어 처리, 손실이 많이 표현되는 데이터에서 사용

In [ ]:
# MNIST 데이터 사용 -> 숫자 이미지
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Flatten, Dense

# * 데이터 준비
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#   - 전처리 (정규화, 인코딩)
#     독립변수 데이터 -> 정규화 (0~255 -> 0~1)
x_train = x_train / 255.0
x_test = x_test / 255.0
#     종속변수 데이터 -> 인코딩 (범주형 데이터 -> 0 ~ 1)
#     0 ~ 9 숫자
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print(x_train.shape, y_train.shape)

# 테스트 옵티마이저 목록
optimizers = ['adam', 'sgd', 'rmsprop']

for opt in optimizers:
  print(f'* ----- {opt} ----- *')

  # * 모델 정의
  model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
  ])
  # 'softmax' : 전체 합이 1이 되도록 0 ~ 1 사이 값으로 바꿔줌
  # Dense(128) 설정 이유?
  # * 보통 32, 64, 128, 256, 512, ... 와 같이 2의 제곱 계열 숫자를 많이 씀
  # * Flatten -> 784개 -> Dense(128) -> Dense(64) -> Dense(10)

  # * 컴파일 : 옵티마이저, 손실함수, 결과 설정
  model.compile(optimizer=opt,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  
  # * 학습
  model.fit(x_train, y_train, epochs=2, batch_size=128, verbose=0)
  # * batch_size : 전체 데이터를 한 번에 학습하지 않고, 일정 개수만큼 끊어서 학습시키는 단위
  #   => 전체 데이터를 한 번에 학습하면 메모리가 감당하기 어려울 수 있음
  #   => 나누어 계산하면 안정적이고 빠름 (128, 256 값을 많이 사용함)
  # [참고] 가중치 업데이트 횟수 : ( total_data_size / batch_size ) * epochs

  # * 평가
  loss, acc = model.evaluate(x_test, y_test)
  print(f' 결과 :: loss - {loss:.4f} / accuracy - {acc:.4f}')
  # => sgd가 다른 옵티마이저에 비해 좋지 않음! 손실값이 3배정도 차이남..

(60000, 28, 28) (60000, 10)
* ----- adam ----- *


c:\Users\user1\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 817us/step - accuracy: 0.9618 - loss: 0.1234
 결과 :: loss - 0.1234 / accuracy - 0.9618
* ----- sgd ----- *
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 764us/step - accuracy: 0.8926 - loss: 0.4023
 결과 :: loss - 0.4023 / accuracy - 0.8926
* ----- rmsprop ----- *
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step - accuracy: 0.9680 - loss: 0.1111
 결과 :: loss - 0.1111 / accuracy - 0.9680
